In [ ]:
!pip install nltk
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

In [ ]:
import pandas as pd
import re

#first test round (sufficient)
annotations_test = pd.read_csv('DataFrame_PS_20240628.csv')
sample_selec_test = pd.read_csv('iaa_first_round.csv')

In [ ]:
ann1 = pd.read_csv('DataFrame_PS_20240710_ann1.csv')
ann2 = pd.read_csv('DataFrame_PS_20240710_ann2.csv')
ann1['annotator'] = 'annotator1'
ann2['annotator'] = 'annotator2'

sample_selec1 = pd.read_csv('ann1_first_round.csv')
sample_selec2 = pd.read_csv('ann2_first_round.csv')
sample_selec = pd.concat([sample_selec1, sample_selec2, sample_selec_test], ignore_index=True)

In [ ]:
note_nrs_1 = list(sample_selec1['note_nr'].unique()) + list(sample_selec_test['note_nr'].unique())
note_nrs_2 = list(sample_selec2['note_nr'].unique()) + list(sample_selec_test['note_nr'].unique())

In [ ]:
print(len(sample_selec))

In [ ]:
annotations_test['note_nr'] = annotations_test['source_file'].replace(['iaa-file105', 'iaa-file134', 'iaa-file143', 'iaa-file15', 'iaa-file153', 'iaa-file161', 'iaa-file172', 'iaa-file186', 'iaa-file202', 'iaa-file26', 'iaa-file70', 'iaa-file85', 'iaa-file88', 'iaa-file96' ], [130897, 36857, 138917, 40048, 42201, 49589, 59783, 59940, 66374, 42295, 113107, 150060, 114600, 130336])

In [ ]:
#ann1['note_nr'] = ann1['source_file'].str.extract(r'(\d+)')
#ann2['note_nr'] = ann2['source_file'].str.extract(r'(\d+)')
def extract_note_nr(text):
    matches = re.findall(r'\d+', text)
    return matches[-1] if matches else None

ann1['note_nr'] = ann1['source_file'].apply(extract_note_nr)
ann1['note_nr'] = ann1['note_nr'].astype(int)

ann2['note_nr'] = ann2['source_file'].apply(extract_note_nr)
ann2['note_nr'] = ann2['note_nr'].astype(int)

In [ ]:
annotations = pd.concat([ann1, ann2, annotations_test], ignore_index = True)

In [ ]:
annotations = annotations.dropna(subset=['note_nr'])
sample_selec = sample_selec.dropna(subset=['note_nr'])

In [ ]:
annotations = annotations.dropna(subset=['note_nr'])
sample_selec = sample_selec.dropna(subset=['note_nr'])

In [ ]:
# Create new '_sentence_text' column and fill with snippets
def get_snippet(row):
    note_nr = row['note_nr']
    begin = int(row['begin']) - 60
    end = int(row['end']) - 60
    
    # Look up the note number in sample_selec
    censored_text = sample_selec[sample_selec['note_nr'] == note_nr]['censored'].values[0]
    
    # Get the snippet from the censored text
    snippet = censored_text[begin:end]
    
    return snippet

# Rename '_sentence_text' to 'inception_sentence'
annotations.rename(columns={'_sentence_text': 'inception_sentence'}, inplace=True)
annotations['_sentence_text'] = annotations.apply(get_snippet, axis=1)

In [ ]:
sample_selec['sentences'] = sample_selec['censored'].apply(lambda x: sent_tokenize(x))

In [ ]:
annotator1 = annotations[annotations['annotator'] == 'annotator1']
annotator2 = annotations[annotations['annotator'] == 'annotator2']

In [ ]:
from fuzzywuzzy import fuzz

def find_match(annotator1, note_nr, sentence, sentence_context, sentences, i, threshold=90):
    def is_fuzzy_match(text1, text2, threshold=90):
        return fuzz.partial_ratio(text1, text2) >= threshold
    
    match = annotator1[(annotator1['note_nr'] == note_nr) & 
                       (annotator1['_sentence_text'].apply(lambda x: is_fuzzy_match(x, sentence)) |
                        annotator1['_sentence_text'].apply(lambda x: is_fuzzy_match(x, sentence_context)) |
                        annotator1['_sentence_text'].apply(lambda x: is_fuzzy_match(sentence, x)) |
                        annotator1['_sentence_text'].apply(lambda x: (
                            is_fuzzy_match(x[:10], sentence) and (i-1 >= 0 and is_fuzzy_match(x[10:], sentences[i-1]))
                        ) or (
                            is_fuzzy_match(x[-10:], sentence) and (i+1 < len(sentences) and is_fuzzy_match(x[:-10], sentences[i+1]))
                        )))]

    return match

In [ ]:
import pandas as pd
import copy
import numpy as np

# Assuming sample_selec and annotator1 are already defined DataFrames

# Copy sample_selec to annotator1_results
annotator1_results = copy.deepcopy(sample_selec)

# Create a new column 'manual_sentence_labels' by copying the 'sentences' column
annotator1_results['manual_sentence_labels'] = annotator1_results['sentences'].copy()

# Iterate over the rows in annotator1_results
for idx, row in annotator1_results.iterrows():
    note_nr = row['note_nr']
    if note_nr in annotator1['note_nr'].values:
        # Get the corresponding sentences for the note_nr
        sentences = row['manual_sentence_labels']
        updated_sentences = []
#         for sentence in sentences:
#             print('sentence:', sentence)
#             match = annotator1[(annotator1['note_nr'] == note_nr) & (annotator1['_sentence_text'] == sentence)]
#             if not match.empty:
#                 updated_sentences.append(match['WHOPS'].values[0])
#             else:
#                 updated_sentences.append(np.nan)
        for i in range(len(sentences)):
            sentence = sentences[i]
            # Initialize an empty list to hold the valid sentences
            context = []
            # Check if sentences[i-1] exists
            if i - 1 >= 0:
                context.append(sentences[i-1])
            # Add sentences[i] (it is assumed that sentences[i] always exists)
            context.append(sentences[i])
            # Check if sentences[i+1] exists
            if i + 1 < len(sentences):
                context.append(sentences[i+1])
            # Join the valid sentences with spaces
            sentence_context = '.'.join(context)
            #print('sentence:', sentence)
            #print('sentence_context', sentence_context)
            # Find the match using the function
            result = find_match(annotator1, note_nr, sentence, sentence_context, sentences, i)

            # Append the corresponding value or np.nan to updated_sentences
            if not result.empty:
                updated_sentences.append(result['WHOPS'].values[0])
            else:
                updated_sentences.append(np.nan)
        # Update the 'manual_sentence_labels' column with the new values
        annotator1_results.at[idx, 'manual_sentence_labels'] = updated_sentences

# Iterate over the rows in annotator1_results again to update non-annotated sentences
for idx, row in annotator1_results.iterrows():
    note_nr = row['note_nr']
    sentences = row['manual_sentence_labels']
    if all(isinstance(s, str) for s in sentences):
        match = annotator1[annotator1['note_nr'] == note_nr]
        if match.empty:
            annotator1_results.at[idx, 'manual_sentence_labels'] = [np.nan] * len(sentences)

# Update 'None' labels to np.nan in 'manual_sentence_labels'
annotator1_results['manual_sentence_labels'] = annotator1_results['manual_sentence_labels'].apply(
    lambda x: [np.nan if v is None else v for v in x]
)

# Replace 'WHO 0', 'WHO 1', etc. with the corresponding integer number in 'manual_sentence_labels'
annotator1_results['manual_sentence_labels'] = annotator1_results['manual_sentence_labels'].apply(
    lambda x: [int(v.split()[1]) if isinstance(v, str) and v.startswith("WHO") else v for v in x]
)

# Copy 'manual_sentence_labels' to 'relevance_manual' and replace values
annotator1_results['relevance_manual'] = annotator1_results['manual_sentence_labels'].apply(
    lambda x: ['irrelevant' if isinstance(v, float) and np.isnan(v) else 'relevant' for v in x]
)









annotator1_results['note_PS_manual'] = np.nan

# Iterate over the rows in annotator1_results
for idx, row in annotator1_results.iterrows():
    note_nr = row['note_nr']
    # Look up the rows in annotator1 with the same note_nr
    matches = annotator1[annotator1['note_nr'] == note_nr]
    # Check if any of these rows contain 'report_type' in the 'sentence_text' column
    report_type_match = matches[matches['inception_sentence'].str.contains('report_type', na=False)]
    if not report_type_match.empty:
        # Assign the value from the 'WHOPS' column of the first matching row to 'note_PS_manual'
        annotator1_results.at[idx, 'note_PS_manual'] = report_type_match['WHOPS'].values[0]

        
        
# Convert 'WHO 0', 'WHO 1', etc. in 'note_PS_manual' to their corresponding integers and ensure NaN values are np.nan
annotator1_results['note_PS_manual'] = annotator1_results['note_PS_manual'].apply(
    lambda x: int(x.split()[1]) if isinstance(x, str) and x.startswith("WHO") else np.nan
)





# Copy the 'note_PS_manual' column and replace values
annotator1_results['relevance_PS_manual'] = annotator1_results['note_PS_manual'].apply(
    lambda x: 'irrelevant' if np.isnan(x) else 'relevant'
)




# Create the new column 'previous_ann' initialized with 'no'
annotator1_results['previous_ann'] = 'no'

# Iterate over the rows in annotator1_results
for idx, row in annotator1_results.iterrows():
    note_nr = row['note_nr']
    # Look up the rows in annotator1 with the same note_nr
    matches = annotator1[annotator1['note_nr'] == note_nr]
    # Check if any of these rows contain 'WHO PS already annotated' in the 'WHOPScat' column
    if any(matches['WHOPScat'].str.contains('WHO PS already annotated', na=False)):
        annotator1_results.at[idx, 'previous_ann'] = 'yes'



In [ ]:
import pandas as pd
import copy
import numpy as np

# Assuming sample_selec and annotator2 are already defined DataFrames

# Copy sample_selec to annotator2_results
annotator2_results = copy.deepcopy(sample_selec)

# Create a new column 'manual_sentence_labels' by copying the 'sentences' column
annotator2_results['manual_sentence_labels'] = annotator2_results['sentences'].copy()

# Iterate over the rows in annotator2_results
for idx, row in annotator2_results.iterrows():
    note_nr = row['note_nr']
    if note_nr in annotator2['note_nr'].values:
        # Get the corresponding sentences for the note_nr
        sentences = row['manual_sentence_labels']
        updated_sentences = []
#         for sentence in sentences:
#             print('sentence:', sentence)
#             match = annotator1[(annotator1['note_nr'] == note_nr) & (annotator1['_sentence_text'] == sentence)]
#             if not match.empty:
#                 updated_sentences.append(match['WHOPS'].values[0])
#             else:
#                 updated_sentences.append(np.nan)
        for i in range(len(sentences)):
            sentence = sentences[i]
            # Initialize an empty list to hold the valid sentences
            context = []
            # Check if sentences[i-1] exists
            if i - 1 >= 0:
                context.append(sentences[i-1])
            # Add sentences[i] (it is assumed that sentences[i] always exists)
            context.append(sentences[i])
            # Check if sentences[i+1] exists
            if i + 1 < len(sentences):
                context.append(sentences[i+1])
            # Join the valid sentences with spaces
            sentence_context = '.'.join(context)
            #print('sentence:', sentence)
            #print('sentence_context', sentence_context)
            # Find the match using the function
            result = find_match(annotator2, note_nr, sentence, sentence_context, sentences, i)

            # Append the corresponding value or np.nan to updated_sentences
            if not result.empty:
                updated_sentences.append(result['WHOPS'].values[0])
            else:
                updated_sentences.append(np.nan)
        # Update the 'manual_sentence_labels' column with the new values
        annotator2_results.at[idx, 'manual_sentence_labels'] = updated_sentences

# Iterate over the rows in annotator2_results again to update non-annotated sentences
for idx, row in annotator2_results.iterrows():
    note_nr = row['note_nr']
    sentences = row['manual_sentence_labels']
    if all(isinstance(s, str) for s in sentences):
        match = annotator2[annotator2['note_nr'] == note_nr]
        if match.empty:
            annotator2_results.at[idx, 'manual_sentence_labels'] = [np.nan] * len(sentences)

# Update 'None' labels to np.nan in 'manual_sentence_labels'
annotator2_results['manual_sentence_labels'] = annotator2_results['manual_sentence_labels'].apply(
    lambda x: [np.nan if v is None else v for v in x]
)

# Replace 'WHO 0', 'WHO 1', etc. with the corresponding integer number in 'manual_sentence_labels'
annotator2_results['manual_sentence_labels'] = annotator2_results['manual_sentence_labels'].apply(
    lambda x: [int(v.split()[1]) if isinstance(v, str) and v.startswith("WHO") else v for v in x]
)

# Copy 'manual_sentence_labels' to 'relevance_manual' and replace values
annotator2_results['relevance_manual'] = annotator2_results['manual_sentence_labels'].apply(
    lambda x: [0 if isinstance(v, float) and np.isnan(v) else 1 for v in x]
)









annotator2_results['note_PS_manual'] = np.nan

# Iterate over the rows in annotator2_results
for idx, row in annotator2_results.iterrows():
    note_nr = row['note_nr']
    # Look up the rows in annotator2 with the same note_nr
    matches = annotator2[annotator2['note_nr'] == note_nr]
    # Check if any of these rows contain 'report_type' in the 'sentence_text' column
    report_type_match = matches[matches['inception_sentence'].str.contains('report_type', na=False)]
    if not report_type_match.empty:
        # Assign the value from the 'WHOPS' column of the first matching row to 'note_PS_manual'
        annotator2_results.at[idx, 'note_PS_manual'] = report_type_match['WHOPS'].values[0]

        
        
# Convert 'WHO 0', 'WHO 1', etc. in 'note_PS_manual' to their corresponding integers and ensure NaN values are np.nan
annotator2_results['note_PS_manual'] = annotator2_results['note_PS_manual'].apply(
    lambda x: int(x.split()[1]) if isinstance(x, str) and x.startswith("WHO") else np.nan
)





# Copy the 'note_PS_manual' column and replace values
annotator2_results['relevance_PS_manual'] = annotator2_results['note_PS_manual'].apply(
    lambda x: 0 if np.isnan(x) else 1
)




# Create the new column 'previous_ann' initialized with 'no'
annotator2_results['previous_ann'] = 0

# Iterate over the rows in annotator2_results
for idx, row in annotator2_results.iterrows():
    note_nr = row['note_nr']
    # Look up the rows in annotator1 with the same note_nr
    matches = annotator2[annotator2['note_nr'] == note_nr]
    # Check if any of these rows contain 'WHO PS already annotated' in the 'WHOPScat' column
    if any(matches['WHOPScat'].str.contains('WHO PS already annotated', na=False)):
        annotator2_results.at[idx, 'previous_ann'] = 1



In [ ]:
print(annotator1_results)

In [ ]:
annotator1_results = annotator1_results[annotator1_results['note_nr'].isin(note_nrs_1)]
annotator2_results = annotator2_results[annotator2_results['note_nr'].isin(note_nrs_2)]

In [ ]:
print(len(annotator1_results))
print(len(annotator2_results))

In [ ]:
annotator1_backup = copy.deepcopy(annotator1_results)
annotator2_backup = copy.deepcopy(annotator2_results)
print(annotator1_results['censored'])
print(annotator1_results[['relevance_PS_manual', 'relevance_manual']].head())

In [ ]:
print(annotator1_backup.head(2)) 

In [ ]:
#annotator1_results['relevance_PS_manual'] = annotator1_results['relevance_manual'].replace({'relevant': 1, 'irrelevant': 0})
#annotator1_results['relevance_manual'] = annotator1_results['relevance_manual'].apply(lambda x: [1 if item == 'relevant' else 0 for item in x])
#annotator1_results['previous_ann'] = annotator1_results['previous_ann'].replace({'relevant': 1, 'irrelevant': 0})

#annotator2_results['relevance_PS_manual'] = annotator2_results['relevance_manual'].replace({'relevant': 1, 'irrelevant': 0})
#annotator2_results['relevance_manual'] = annotator2_results['relevance_manual'].apply(lambda x: [1 if item == 'relevant' else 0 for item in x])

In [ ]:

print(annotator1_results['censored'])
print(annotator1_results[['relevance_PS_manual', 'relevance_manual']].head())

In [ ]:
print(annotator1_results.head(2))

In [ ]:
annotator_all = pd.concat([annotator1_results, annotator2_results], ignore_index = False)

In [ ]:
#annotator_all.to_csv('first_round_before_adjudicion.csv')